In [29]:
# Imports and configuration
import pandas as pd
from src.utils.file_utils import load_config, get_path

cfg_clean = load_config("clean")

In [30]:
# Read in raw data file
input_file_path = get_path("hmda_raw")
raw_hmda_df = pd.read_parquet(input_file_path)
print(raw_hmda_df.shape)

(12236879, 85)


In [31]:
# Drop columns that are not known prior to application except for target
features_to_drop = cfg_clean["features"]["drop"]
raw_hmda_df.drop(columns=features_to_drop, inplace=True)
print(raw_hmda_df.shape)

(12236879, 67)


In [34]:
# Strip leading and trailing whitespace
# All columns are currently string values, so we will perform this on all columns regardless of eventual datatype
raw_hmda_df = raw_hmda_df.apply(lambda s: s.astype(str).str.strip())